# Financial Ledger #
### Financial Ledger that can construct graphs n' shit ###


In [39]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
CURRENCY_PREFIX = "$NZD"

In [40]:
df_data = pd.read_csv("Financial Ledger (Sheet1).csv")



### Dummy Data Testing ###

In [41]:
dummy_entries = pd.DataFrame([
    {"Date": "4/6/2025 15:30", "Description": "Bus Card Top-Up", "Amount": -20.0, "Category": "Transport"},
    {"Date": "5/6/2025 10:00", "Description": "Freelance Payout", "Amount": 150.0, "Category": "Income"},
    {"Date": "5/6/2025 13:45", "Description": "Boba Tea (L)", "Amount": -6.5, "Category": "Drinks"},
    {"Date": "5/6/2025 18:20", "Description": "Udemy Python Course", "Amount": -12.99, "Category": "Education"},
])
df_data = pd.concat([df_data,dummy_entries],ignore_index=True)
df_data["Date"] = pd.to_datetime(df_data["Date"],format="%d/%m/%Y %H:%M",dayfirst=True)


In [42]:
df_data

,Date,Description,Amount,Category
0,2025-03-28 20:53:00,Boba Tea (L),-6.50,Drinks
1,2025-03-29 16:29:00,Bus Card Top-Up,-20.00,Transport
2,2025-03-31 12:08:00,Taxi Ride,-18.00,Transport
3,2025-04-01 17:11:00,Book Purchase,-13.50,Education
4,2025-04-02 19:27:00,Coffee (M),-4.50,Drinks
...,...,...,...,...
99,2025-07-05 04:36:00,Book Purchase,-13.50,Education
100,2025-06-04 15:30:00,Bus Card Top-Up,-20.00,Transport
101,2025-06-05 10:00:00,Freelance Payout,150.00,Income
102,2025-06-05 13:45:00,Boba Tea (L),-6.50,Drinks


## Core Metrics ##

In [43]:
total_income = df_data[df_data["Amount"] > 0]["Amount"].sum()
total_spent = df_data[df_data["Amount"] < 0]["Amount"].sum()
net_balance = df_data["Amount"].sum()
transaction_count = df_data["Date"].count()
mean_day = df_data[df_data["Amount"] < 0].resample("D",on="Date").sum().query("Description != '0'")["Amount"].mean()
spending_by_category = df_data[df_data["Amount"] < 0].groupby("Category")["Amount"].sum().abs().sort_values(ascending=False)
spending_by_category

Category
Transport        478.00
Education        277.89
Food             271.20
Drinks           150.50
Entertainment     89.91
Name: Amount, dtype: float64

In [44]:
print(f"💸 Total Amount of Money Spent: {CURRENCY_PREFIX}{abs(total_spent):,.2f}")
print(f"💰 Total Income:                {CURRENCY_PREFIX}{total_income:,.2f}")
print(f"🧮 Net Balance:                 {CURRENCY_PREFIX}{net_balance:,.2f}")
print(f"🧾 Transaction Count:           {CURRENCY_PREFIX }{transaction_count}")
print(f"🧾 Average Spending Per Day:    {CURRENCY_PREFIX}{abs(mean_day)}")


💸 Total Amount of Money Spent: $NZD1,267.50
💰 Total Income:                $NZD2,800.00
🧮 Net Balance:                 $NZD1,532.50
🧾 Transaction Count:           $NZD104
🧾 Average Spending Per Day:    $NZD12.675


## Graphs ##

#### Spending by Category ####

In [45]:
spending_by_category_line = px.bar(x=spending_by_category.index,y=spending_by_category.values,color=spending_by_category.index,
                                   title="Spending By Category",hover_name=spending_by_category.index)
spending_by_category_line.update_layout(xaxis_title = "Category",yaxis_title = f"Amount ({CURRENCY_PREFIX})")

In [ ]:
spending_by_day = df_data[df_data["Amount"] < 0].resample("D",on="Date")..query("Description != 0").reset_index()
spending_by_day["Amount"] = spending_by_day["Amount"].abs()
spending_by_day["RollingMean"] = spending_by_day["Amount"].rolling(window=5).mean()
spending_by_day_roll = spending_by_day["Amount"].rolling(window=5).mean()
spending_by_day_line = px.scatter(spending_by_day,x="Date",y="RollingMean",title = "Total Daily Spending",
                               trendline="ols",trendline_color_override = "red",hover_name="Description",
                               hover_data=["Category", "Amount"])#The small info in the hover box
spending_by_day_line.update_layout(xaxis_title = "Category",yaxis_title = f"Amount ({CURRENCY_PREFIX})",
                                   xaxis = dict(tickangle=45),)

UndefinedVariableError: name 'Description' is not defined

In [ ]:
spending_by_week = (
    df_data[df_data["Amount"] < 0]
    .resample("W", on="Date")
    .mean()
    .reset_index()
)
spending_by_week["Amount"] = spending_by_week["Amount"].abs()
spending_by_week["RollingMean"] = spending_by_week["Amount"].rolling(window=1).mean() #Change later when the data becomes huge, useless for now

fig = px.scatter(
    spending_by_week,
    x="Date",
    y="RollingMean",
    title="Total Weekly Spending (5-Week Avg)",
    trendline="ols",
    trendline_color_override="red",
    hover_data={"Amount": True},
)

fig.update_layout(
    xaxis_title="Week",
    yaxis_title=f"Amount ({CURRENCY_PREFIX})",
    xaxis=dict(tickangle=45),
)


In [56]:
spending_by_day

,Amount,RollingMean
Date,,
2025-03-28,6.5,NaN
2025-03-29,20.0,NaN
2025-03-30,0.0,NaN
2025-03-31,18.0,NaN
2025-04-01,13.5,11.600
...,...,...
2025-07-01,0.0,12.538
2025-07-02,45.2,18.980
2025-07-03,4.5,10.840
